In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
#import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sb
from collections import Counter
import matplotlib as mpl
from sklearn.model_selection import train_test_split
from sklearn.ensemble import VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import SGDClassifier 
%matplotlib inline

In [ ]:
#load our data
df=pd.read_csv('../input/students-performance-in-exams/StudentsPerformance.csv')
df.head()

In [ ]:
#let's take alook about some information as type and non values
df.info()

As we see we have no nulls in our data

# Adding new column which combine the final score of reading ,writing, and math 

In [ ]:
df.columns

In [ ]:
#adding reading,writing,math to total_score
df['total_score']=df['math score']+df['reading score']+df['writing score']

In [ ]:
df.head()

In [ ]:
#shape of df
df.shape

# Number of males and femals

In [ ]:
plt.figure(figsize=[15,5])
plt.subplot(1,2,1)
plt.pie(df['gender'].value_counts(),labels=df['gender'].value_counts().index,explode=[.1,.1],autopct='%1.1f%%',shadow=True);
plt.title('percentage of males and females')
plt.subplot(1,2,2)
#base_color = sb.color_palette()[5]
sb.countplot(data=df,x='gender' )
plt.title('Number of males and females')

Distribution of our numerical features with each other

In [ ]:
#Distribution of our numerical features with each other
sb.pairplot(df, hue="gender")

# Numbers of each gender in each race

In [ ]:
#Numbers of each gender in each race
sb.set(style='darkgrid')
plt.figure(figsize=[10,5])
ax=sb.countplot(data=df,x='gender',hue='race/ethnicity')
plt.title('Numbers of each gender in each race');

# Distribution of total score for each gender in each group

In [ ]:
#Distribution of total score for each gender in each group
plt.figure(figsize=[10,10]);
sb.set_theme(style="darkgrid")
sb.displot(df, x="total_score", col="race/ethnicity", row="gender",
             binwidth=3, height=3, facet_kws=dict(margin_titles=True),);
 

# Number of each gender in math score

In [ ]:
f, ax = plt.subplots(figsize=(10, 5))
sb.despine(f)

sb.histplot(
    df,
    x="math score", hue="gender",
    multiple="stack",
    palette="light:m_r",
    edgecolor=".3",
    linewidth=.5,
    log_scale=False,)
plt.title('Number of each gender in math score')
ax.xaxis.set_major_formatter(mpl.ticker.ScalarFormatter());
ax.set_xticks([0,10,20,30,40,50,60,70,80,90,100,110]);

In [ ]:
sb.set(style='darkgrid')
plt.figure(figsize=[10,5])
ax=sb.countplot(data=df,x='parental level of education',hue='gender')
plt.xticks(rotation=45)
plt.title('Numbers of each gender in each level education');

# Number in each race which has test preparation course

In [ ]:
#Number in each race which has test preparation course
plt.figure(figsize=[10,7])
counts = df.groupby(['race/ethnicity', 'test preparation course']).size()
counts = counts.reset_index(name='count')
# Use DataFrame.pivot() to rearrange the data, to have race class on rows
counts =counts.pivot(index = 'race/ethnicity', columns = 'test preparation course' ,values = 'count')
#drow hwat map 
sb.heatmap(counts,annot = True, fmt = 'd');
#get the title
plt.title('Number in each race which has test preparation course');

# Math score statistics for each gender

In [ ]:
#math score statistics for each gender
base_color = sb.color_palette()[4]
sb.violinplot(data=df, x='gender', y='math score', color=base_color, innner=None)
plt.xticks(rotation=15);


WE see that the main of males is higher than femals

# Writing score statistics for each gender

In [ ]:
base_color = sb.color_palette()[4]
sb.boxplot(data=df, x='gender', y='writing score', color=base_color )
plt.xticks(rotation=15);

In [ ]:
# Draw a categorical scatterplot to show each observation
plt.figure(figsize=[10,6])
ax = sb.swarmplot(data=df, x="reading score", y="gender",hue='lunch');
ax.set(ylabel="");

# Total score with types of races

In [ ]:
#plotting boxplot for Total score with types of races
sb.set_theme(style="ticks")

# Initialize the figure with a logarithmic x axis
f, ax = plt.subplots(figsize=(7, 6))
ax.set_xscale("log")

sb.boxplot(x="total_score", y="race/ethnicity", data=df,
            whis=[0, 100], width=.6, palette="vlag")

# Add in points to show each observation
sb.stripplot(x="total_score", y="race/ethnicity", data=df,
              size=2, color=".3", linewidth=0)

# Tweak the visual presentation
plt.xticks( )
ax.xaxis.grid(False)
ax.set(ylabel="")
sb.despine(trim=False, left=True)

# Anomaly Detection

In [ ]:

def detect_outliers(df,features):
    outlier_indices = []
    
    for c in features:
        # 1st quartile
        Q1 = np.percentile(df[c],25)
        # 3rd quartile
        Q3 = np.percentile(df[c],75)
        # IQR
        IQR = Q3 - Q1
        # Outlier step
        outlier_step = IQR * 1.5
        # detect outlier and their indeces
        outlier_list_col = df[(df[c] < Q1 - outlier_step) | (df[c] > Q3 + outlier_step)].index
        # store indeces
        outlier_indices.extend(outlier_list_col)
    
    outlier_indices =Counter(outlier_indices)
    multiple_outliers = list(i for i, v in outlier_indices.items() if v > 2)
    
    return multiple_outliers

In [ ]:
df.loc[detect_outliers(df,['math score', 'reading score', 'writing score'])]

In [ ]:
# drop outliers
df =df.drop(detect_outliers(df,['math score', 'reading score', 'writing score']),axis = 0).reset_index(drop = True)

# Relation between our features

first we will replace each catoegory with unique number to make it easy to deel with our feature

In [ ]:
#encoding gender feature
df['gender']=df['gender'].replace({'female':0,'male':1})

In [ ]:
#we want to know the uniques in race
df['race/ethnicity'].unique()

In [ ]:
# encoding race
df['race/ethnicity']=df['race/ethnicity'].replace({'group B':1, 'group C':2, 'group A':3, 'group D':4, 'group E':5})

In [ ]:
# uniques of parental level of education
df['parental level of education'].unique()


In [ ]:
# encoding parental level of education
df['parental level of education']=df['parental level of education'].replace({"bachelor's degree":1, 'some college':2, "master's degree":3,
       "associate's degree":4, 'high school':5, 'some high school':6})

In [ ]:
# uniques of lunch
df['lunch'].unique()

In [ ]:
# encoding lunch
df['lunch']=df['lunch'].replace({'standard':1, 'free/reduced':0})

In [ ]:
# uniques of test preparation course
df['test preparation course'].unique()

In [ ]:
# encoding test preparation course 
df['test preparation course']=df['test preparation course'].replace({'none':0, 'completed':1})

# Our new encoded data NOW

let's take alook after we encode our data

In [ ]:
df.head()

In [ ]:
# drawing heatmap
plt.figure(figsize=[10,10])
sb.heatmap(df.corr(),annot=True, cmap='Dark2_r', linewidths = .2)
plt.title('Relation between variables');

As we see above the three scores in writing ,reading and math have strong relation between each other

# Now we will make prediction of gender based on other features

Because od the strong relations between these three features we will drop two of them

In [ ]:
df.drop(['writing score','reading score'],inplace=True,axis=1)


In [ ]:
X=df.drop(['gender'],axis=1)
y=df['gender']

Rescale total score and math score

In [ ]:
X['total_score']=X['total_score']/X['total_score'].max()
X['math score']=X['math score']/X['math score'].max()


In [ ]:
#Splitting data

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=44, shuffle =True)

#Splitted Data
print('X_train shape is ' , X_train.shape)
print('X_test shape is ' , X_test.shape)
print('y_train shape is ' , y_train.shape)
print('y_test shape is ' , y_test.shape)

In [ ]:
#Applying VotingClassifier Model 

#loading models for Voting Classifier
LRModel_ = LogisticRegression(solver='lbfgs', multi_class='multinomial',random_state=33)
DTModel_ = DecisionTreeClassifier(criterion = 'entropy',max_depth=20,random_state = 33)
KNNModel_ = KNeighborsClassifier(n_neighbors= 20 , weights ='uniform', algorithm='auto')
SGDModel_ = SGDClassifier(loss='log', penalty='l2', max_iter=200, tol=1e-5)

#loading Voting Classifier
VotingClassifierModel = VotingClassifier(estimators=[('LRModel',LRModel_),('DTModel',DTModel_),('KNNModel',KNNModel_),('SGDModel',SGDModel_)],n_jobs=10 ,voting='hard')
VotingClassifierModel.fit(X_train, y_train)

#Calculating Details
print('VotingClassifierModel Train Score is : ' , VotingClassifierModel.score(X_train, y_train))
print('VotingClassifierModel Test Score is : ' , VotingClassifierModel.score(X_test, y_test))
print('----------------------------------------------------')

#Calculating Prediction
y_pred = VotingClassifierModel.predict(X_test)
print('Predicted Value for VotingClassifierModel is : ' , y_pred[:10])

In [ ]:
#Import Libraries
from sklearn.metrics import accuracy_score
 
#Calculating Accuracy Score  : ((TP + TN) / float(TP + TN + FP + FN))
AccScore = accuracy_score(y_test, y_pred, normalize=True)
print('Accuracy Score is : ', AccScore)

#  Please if you benefit from this kernal UPVOTE it 